In [178]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [179]:
df = pd.read_csv('./_data.csv', sep=',')
includes = []
for i in range(len(df)):
    if 'Москва' not in df['Адрес'][i]:
        includes.append(i)
print(df.columns.to_list())
df_clean=df.drop(includes)
print(len(df_clean))
df = df_clean.copy().drop(['Тип', 'Unnamed: 0', 'Телефоны', 'Серия дома', 'Название ЖК', 'Ссылка на объявление','Описание'], axis=1)
df['Ремонт'].fillna('неизвестно')

df.to_csv('data.csv', sep='\t')

['Unnamed: 0', 'ID  объявления', 'Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны', 'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод', 'Ссылка на объявление']
19737


In [180]:
df.head(2) 


,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)",Да
1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)",Нет


In [181]:
#Приведим все комнаты в числовое или около того значение
df['Количество комнат'] = df['Количество комнат'].str[0]


In [182]:
#Убираем NaN в кол-ве комнат
df['Количество комнат'].fillna(int(df['Количество комнат'].mode()), inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_12548\1043790658.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  df['Количество комнат'].fillna(int(df['Количество комнат'].mode()), inplace=True)


In [183]:
# Считаем, что отдельные площади кухни и комнат нам не важны, и оставляет только общую площадь.
df['Площадь кв, м2'] = [row.split('/')[0] for row in df['Площадь, м2']]
# И убираем две колонки с площадью
# Удаляем колонки со старой площадью
df = df.drop(['Площадь, м2'], axis = 1)
df = df.drop(['Площадь комнат, м2'], axis = 1)

In [184]:
# Убираем NaN в колонке 'Окна'
df['Окна'] = df['Окна'].fillna('Во двор')

In [185]:
# Убираем NaN в колонке 'Ремонт'
df['Ремонт'] = df['Ремонт'].fillna('Косметический')

In [186]:
# Вытаскиваем информацию из колонки 'дом'
df['Тип дома'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
df['Этажность дома'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['Этаж квартиры'] = [int(row.split('/')[0]) for row in df['Дом']]
df = df.drop(['Дом'], axis = 1)

In [187]:
# Избавляемся от NaN для лифтов, если меньше 5 этажей - нет лифта, в остальном по одному каждого лифта
df['Лифт'] = [row if row == row else
              ['Пасс (1), Груз (1)' if df['Этажность дома'].iloc[i]>5 else 'Пасс (0), Груз (0)'][0]
              for i, row in enumerate(df['Лифт'])]

In [188]:
# Убираем NaN в колонке 'Мусоропровод'
df['Мусоропровод'] = df['Мусоропровод'].fillna('Нет')

In [189]:
# Убираем NaN в колонке 'Высота потолков, м'
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(round(df['Высота потолков, м'].mean(), 2))


In [190]:
# Убираем NaN в колонке 'Санузел'
df['Санузел'] = df['Санузел'].fillna('Совмещенный (1)')
df['Санузел совмещенный'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Совмещенный' else '(0)' for row in df['Санузел']]
df['Санузел раздельный'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Раздельный' else '(0)' for row in df['Санузел']]
df['Санузел совмещенный'] = [row.split('(')[1] for row in df['Санузел совмещенный']]
df['Санузел совмещенный'] = [int(row.split(')')[0]) for row in df['Санузел совмещенный']]
df['Санузел раздельный'] = [row.split('(')[1] for row in df['Санузел раздельный']]
df['Санузел раздельный'] = [int(row.split(')')[0]) for row in df['Санузел раздельный']]
df = df.drop(['Санузел'], axis = 1)

In [191]:
# Убирам NaN в "Дополнительно"
df['Дополнительно'] = df['Дополнительно'].fillna('Пусто')

In [192]:
# Убираем NaN в колонке 'Балкон'
df['Балкон'] = df['Балкон'].fillna('Балкон (0), Лоджия (0)')
df['Лоджия'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Лоджия' else '(0)' for row in df['Балкон']]
df['Балкон'] = [row.split(' ')[1] if row.split(' ')[0] == 'Балкон' else '(0)' for row in df['Балкон']]
df['Лоджия'] = [row.split('(')[1] for row in df['Лоджия']]
df['Лоджия'] = [int(row.split(')')[0]) for row in df['Лоджия']]
df['Балкон'] = [row.split('(')[1] for row in df['Балкон']]
df['Балкон'] = [int(row.split(')')[0]) for row in df['Балкон']]

In [193]:
df['Парковка'] = df['Парковка'].fillna('наземная')

In [195]:
df['Можно с детьми/животными']=df['Можно с детьми/животными'].fillna('по договорённости')

In [196]:
df.columns.to_list()

['ID  объявления',
 'Количество комнат',
 'Метро',
 'Адрес',
 'Парковка',
 'Цена',
 'Ремонт',
 'Балкон',
 'Окна',
 'Можно с детьми/животными',
 'Дополнительно',
 'Высота потолков, м',
 'Лифт',
 'Мусоропровод',
 'Площадь кв, м2',
 'Тип дома',
 'Этажность дома',
 'Этаж квартиры',
 'Санузел совмещенный',
 'Санузел раздельный',
 'Лоджия']

In [197]:
#Переводим расстояние до метро в метры в зависимости от указанного способо передвижения, для NaN задаём медиану
df['время до метро'] = df['Метро'].str.replace(')', '').str.split('(').str[-1]
df[['время в минутах', 'способ']] = df['время до метро'].str.extract(r'(\d+) мин\s(пешком|на машине)')
df['способ'] = df['способ'].apply(lambda x: 70 if x == 'пешком' else 750)
df['способ'] = df['способ'].fillna(0)
df['время в минутах'] = df['время в минутах'].fillna(0)
df['distance to subway, m'] = df['способ'] * df['время в минутах'].astype(int)
median_value = df['distance to subway, m'].median()
df['distance to subway, m'] = df['distance to subway, m'].replace(0, median_value)


In [198]:
df.rename(columns={'ID  объявления': 'Ad_ID'}, inplace=True)
df.rename(columns={'Количество комнат': 'Number_of_Rooms'}, inplace=True)
df.rename(columns={'Метро': 'Metro'}, inplace=True)
df.rename(columns={'Адрес': 'Address'}, inplace=True)
df.rename(columns={'Парковка': 'Parking'}, inplace=True)
df.rename(columns={'Цена': 'Price'}, inplace=True)
df.rename(columns={'Ремонт': 'Renovation'}, inplace=True)
df.rename(columns={'Балкон': 'Balcony'}, inplace=True)
df.rename(columns={'Окна': 'Windows'}, inplace=True)
df.rename(columns={'Можно с детьми/животными': 'Children/Pets_Allowed'}, inplace=True)
df.rename(columns={'Дополнительно': 'Additional Info'}, inplace=True)
df.rename(columns={'Высота потолков, м': 'Ceiling_Height_M'}, inplace=True)
df.rename(columns={'Лифт': 'Elevator'}, inplace=True)
df.rename(columns={'Мусоропровод': 'Garbage_Chute'}, inplace=True)
df.rename(columns={'Площадь кв, м2': 'Area_sq_m'}, inplace=True)
df.rename(columns={'Тип дома': 'House_Type'}, inplace=True)
df.rename(columns={'Этажность дома': 'Number_of_Floors'}, inplace=True)
df.rename(columns={'Этаж квартиры': 'Floor_of_Apartment'}, inplace=True)
df.rename(columns={'Санузел совмещенный': 'Combined_Bathroom'}, inplace=True)
df.rename(columns={'Санузел раздельный': 'Separate_Bathroom'}, inplace=True)
df.rename(columns={'Лоджия': 'Balconette'}, inplace=True)
df.drop(['время до метро'], axis=1, inplace = True)
df.drop(['время в минутах'], axis=1, inplace = True)
df.drop(['способ'], axis=1, inplace = True)
df.drop(['Metro'], axis=1, inplace = True)

In [112]:
df.head(10) 

,Ad_ID,Number_of_Rooms,Metro,Address,Parking,Price,Renovation,Balcony,Windows,Children/Pets_Allowed,...,Elevator,Garbage_Chute,Area_sq_m,House_Type,Number_of_Floors,Floor_of_Apartment,Combined_Bathroom,Separate_Bathroom,Balconette,"subway in, m"
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,Во двор,"Можно с детьми, Можно с животными",...,"Пасс (4), Груз (1)",Да,200.0,Монолитный,16,5,1,0,0,900
1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,На улицу и двор,Можно с детьми,...,"Пасс (1), Груз (1)",Нет,198.0,Монолитно-кирпичный,16,5,2,1,0,800
2,271173086,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,0,На улицу и двор,Можно с детьми,...,Пасс (1),Нет,200.0,Неизвестно,16,5,3,0,0,700
3,272197456,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,0,На улицу и двор,Можно с животными,...,Пасс (1),Нет,170.0,Неизвестно,6,5,3,0,0,300
4,273614615,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",наземная,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,0,На улицу и двор,Можно с детьми,...,"Пасс (1), Груз (1)",Да,58.0,Панельный,26,12,2,0,0,700
5,274837728,3,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",наземная,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",Дизайнерский,0,Во двор,Можно с детьми,...,"Пасс (1), Груз (1)",Нет,92.0,Неизвестно,7,3,1,0,0,500
6,273643908,"5, Оба варианта",м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",Евроремонт,0,На улицу и двор,"Можно с детьми, Можно с животными",...,Пасс (1),Нет,213.0,Неизвестно,5,4,2,0,0,400
7,274475342,3,м. Смоленская (3 мин пешком),"Москва, улица Арбат, 43С3",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Евроремонт,0,Во двор,"Можно с детьми, Можно с животными",...,Пасс (1),Нет,98.0,Монолитный,4,2,1,1,0,300
8,273973191,3,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Евроремонт,0,На улицу,Можно с животными,...,Пасс (1),Нет,120.0,Сталинский,10,5,1,0,0,900
9,272900409,4,м. Арбатская (10 мин пешком),"Москва, улица Арбат, 30/3С1",открытая,"210000.0 руб./ За месяц, Залог - 210000 руб., ...",Евроремонт,1,На улицу и двор,Можно с детьми,...,Пасс (1),Да,90.0,Сталинский,7,2,1,1,0,1000


In [199]:
nans = df.isna().any().any()
print(nans)


False


In [20]:
df.to_csv('final.csv', sep=',')